# Workshop #1 Data Cleaning
## USS x WIDS DataVerse

###  Why Data Cleaning Matters??
Data cleaning ensures your dataset is accurate, consistent, and usable. Real-world data often comes with missing values, inconsistent labels, mixed units, and messy date formats. If we skip cleaning, even simple analyses can become misleading.

###  Dangerous Consequences of Messy Data

Imagine a hospital dataset where weights are mixed: some in **kg**, some in **lb**.  

If unchecked, `70 lb` could be treated as `70 kg` when calculating drug doses **doubling the intended amount**.  

Messy data like this may seem minor, but it can lead to **serious, real-world harm**.  
Data cleaning **isn’t** optional.


## Get Started!
In this notebook, we’ll go through a practical data cleaning workflow in R using a small dummy dataset (30 rows). The dataset contains:

- **Height** split into numeric value and unit (`cm` / `mm`)  
- **Weight** mostly as integers, with some missing values  
- **Age** with minor datatype issues (e.g., strings instead of numbers)  
- **Dates** in mixed simple formats  
- **Categories** with inconsistent capitalization (`A/a/B/b/C`)  

We’ll follow these steps to clean the data:

1. Import the dataset  
2. Handle missing values  
3. Correct datatypes  
4. Parse dates  
5. Rename columns  
6. Standardize units (convert height from mm → cm)  
7. Produce the final cleaned dataset


## 0. Load packages

First we load the R packages we'll use throughout the workshop:

- **tidyverse** for data import and wrangling
- **lubridate** for parsing and working with dates
If these packages are not installed on your machine, uncomment the `install.packages()` line and run it once.


In [7]:
# Install once if needed:
# install.packages(c("tidyverse", "lubridate", "skimr"))

library(tidyverse)
library(lubridate)

## 1. Import data

We read the CSV file into R using `readr::read_csv()`. This gives us a tibble (a modern data frame) and a quick preview of the data.

We'll also:

- Look at the first few rows
- Check the structure of the dataset
- Get a high-level summary including missing values


In [8]:
patients_raw <- read_csv("simple_dummy_data.csv")

head(patients_raw)
glimpse(patients_raw)
summary(patients_raw)

Rows: 30 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, height_unit, date_of_visit, category
dbl (5): id, age, height_value, weight_kg, score

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,name,age,height_value,height_unit,weight_kg,date_of_visit,category,score
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Alice,NA,1650,mm,62,2025-01-23,a,57
2,Bob,28,167,cm,81,2025-01-17,B,42
3,Charlie,24,173,cm,81,01/02/2025,NA,57
4,David,28,1705,mm,86,01/16/2025,B,41
5,Eva,25,1704,mm,64,2025-01-09,NA,200
6,Frank,NA,178,cm,NA,2025-01-03,B,200


Rows: 30
Columns: 9
$ id            <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1…
$ name          <chr> "Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Gra…
$ age           <dbl> NA, 28, 24, 28, 25, NA, 29, 49, 29, 29, 44, 45, 47, 36, …
$ height_value  <dbl> 1650, 167, 173, 1705, 1704, 178, 164, 1500, 1740, 1506, …
$ height_unit   <chr> "mm", "cm", "cm", "mm", "mm", "cm", "cm", "mm", "mm", "m…
$ weight_kg     <dbl> 62, 81, 81, 86, 64, NA, 56, NA, 55, 46, 60, 77, 82, 76, …
$ date_of_visit <chr> "2025-01-23", "2025-01-17", "01/02/2025", "01/16/2025", …
$ category      <chr> "a", "B", NA, "B", NA, "B", "a", "b", "b", "C", "a", "B"…
$ score         <dbl> 57, 42, 57, 41, 200, 200, 48, 55, 200, 42, 52, 46, 52, 4…


       id            name                age         height_value   
 Min.   : 1.00   Length:30          Min.   :19.00   Min.   : 150.0  
 1st Qu.: 8.25   Class :character   1st Qu.:27.00   1st Qu.: 158.5  
 Median :15.50   Mode  :character   Median :29.00   Median : 175.5  
 Mean   :15.50                      Mean   :33.04   Mean   : 670.7  
 3rd Qu.:22.75                      3rd Qu.:37.00   3rd Qu.:1581.0  
 Max.   :30.00                      Max.   :49.00   Max.   :1802.0  
                                    NA's   :5                       
 height_unit          weight_kg     date_of_visit        category        
 Length:30          Min.   :46.00   Length:30          Length:30         
 Class :character   1st Qu.:60.25   Class :character   Class :character  
 Mode  :character   Median :74.00   Mode  :character   Mode  :character  
                    Mean   :70.15                                        
                    3rd Qu.:81.00                                        
    

## 2. Handle Missing Values

Before performing most analyses, we need to **identify and handle missing values (`NA`)**.

In this dataset:

- `age` may contain non-numeric strings (e.g., `"None"`) that should be treated as missing  
- `weight_kg` has some missing values  
- `category` has missing values  
- `score` has missing values  

### 🔹 Simple Approach (for this workshop)
For simplicity, we will **remove rows with missing values**.  

- This is easy to understand and demonstrates the effect of missing data  
- However, in real projects, **deleting data can lead to bias** if the missingness is not random  

### 🔹 Other Common Approaches

1. **Imputation with Mean or Median**
   - Replace missing numeric values with the **mean** or **median** of that variable  
   - **When to use:**  
     - Mean: when data is roughly normally distributed  
     - Median: when data is skewed or contains outliers  
   - Example: replacing missing `weight_kg` with the average weight  

2. **Mode Imputation**
   - Replace missing **categorical values** with the most frequent category  
   - **When to use:** if one category dominates and missingness is small  

3. **Special Values**
   - Sometimes missing values are meaningful and can be encoded explicitly (e.g., `"Unknown"`)  
   - **When to use:** if missingness itself carries information  



> **Tip:** The choice depends on **why the data is missing** and **how much is missing**.  
> Understanding the pattern of missingness is critical before deciding on a method.

In [9]:
# Count missing values (and potential issues) in the raw data
colSums(is.na(patients_raw))

# Removing all NA values 
patients_step2 <- na.omit(patients_raw)

# Check missing values after imputation
colSums(is.na(patients_step2))

id          name           age  height_value   height_unit 
            0             0             5             0             0 
    weight_kg date_of_visit      category         score 
            4             2             5             2

id          name           age  height_value   height_unit 
            0             0             0             0             0 
    weight_kg date_of_visit      category         score 
            0             0             0             0

## Give it a try! Short refresher
### 1. Numeric – Salaries
A dataset contains `salary` for all employees in a large company. Some entries are missing.  
**Question:** How would you handle the missing values?

---

### 2. Numeric – Test Scores
A class dataset contains `exam_score` for 100 students. Two scores are missing.  
**Question:** How would you handle the missing values?

---

## 3. Fix datatypes

CSV files often store values with slightly incorrect data types. For example:

- IDs might be read as character instead of integer **(Although it isn't in this case)**
- Categories might be plain text instead of factors (categorical variables)
  
**NOTE: category has 3 levels (A,B,C) but we see a few few entries aren't capitalized (This can be a problem)** 

Here, we:

- Ensure `id` is stored as an integer
- Correctly connvert `category` to a factor
- Ensure 'age' is stored as a numeric data type
- Verify the overall structure of the dataset


In [4]:
# 3. Fix datatypes -------------------------------------------------------

patients_step3 <- patients_step2 %>%
  mutate(
    id       = as.integer(id),
    category = str_to_upper(category), # important addition
    category = as.factor(category),
    age = as.numeric(age)
  )

str(patients_step3)


tibble [15 × 9] (S3: tbl_df/tbl/data.frame)
 $ id           : int [1:15] 2 4 7 9 10 11 12 14 18 19 ...
 $ name         : chr [1:15] "Bob" "David" "Grace" "Ian" ...
 $ age          : num [1:15] 28 28 29 29 29 44 45 36 31 26 ...
 $ height_value : num [1:15] 167 1705 164 1740 1506 ...
 $ height_unit  : chr [1:15] "cm" "mm" "cm" "mm" ...
 $ weight_kg    : num [1:15] 81 86 56 55 46 60 77 76 86 66 ...
 $ date_of_visit: chr [1:15] "2025-01-17" "01/16/2025" "01/19/2025" "01/09/2025" ...
 $ category     : Factor w/ 3 levels "A","B","C": 2 2 1 2 3 1 2 2 1 3 ...
 $ score        : num [1:15] 42 41 48 200 42 52 46 47 58 56 ...
 - attr(*, "na.action")= 'omit' Named int [1:15] 1 3 5 6 8 13 15 16 17 21 ...
  ..- attr(*, "names")= chr [1:15] "1" "3" "5" "6" ...


## 4. Parse mixed-format dates

The `date_of_visit` column contains dates in **mixed formats**, e.g.:

- `"2025-01-23"` (YYYY-MM-DD)
- `"01/16/2025"` (MM/DD/YYYY)

We can't safely work with these as plain text, so we'll parse them into a proper `Date` column called `visit_date`.

We use `lubridate::parse_date_time()` and provide multiple possible date orders (`"ymd"` and `"mdy"`). R will try them in order until it finds one that matches for each value.


In [5]:
# 4. Parse date column ---------------------------------------------------

patients_step4 <- patients_step3 %>%
  mutate(
    visit_date = parse_date_time(
      date_of_visit,
      orders = c("ymd", "mdy")
    ) %>% as_date()
  )

patients_step4 %>%
  select(date_of_visit, visit_date) %>%
  head()


date_of_visit,visit_date
<chr>,<date>
2025-01-17,2025-01-17
01/16/2025,2025-01-16
01/19/2025,2025-01-19
01/09/2025,2025-01-09
01/18/2025,2025-01-18
2025-01-11,2025-01-11


## 5. Rename columns

Clear and consistent column names make your code easier to read and maintain.

We currently have height stored as:

- `height_value` – the numeric value
- `height_unit` – the unit (`"cm"` or `"mm"`)

We'll rename these to:

- `height_raw` – the original numeric height value
- `height_unit_raw` – the original unit

Later, we'll create a clean, standardized `height_cm` variable.


In [6]:
# 5. Rename columns ------------------------------------------------------

patients_step5 <- patients_step4 %>%
  rename(
    height_raw      = height_value,
    height_unit_raw = height_unit
  )

colnames(patients_step5)


[1] "id"              "name"            "age"             "height_raw"     
 [5] "height_unit_raw" "weight_kg"       "date_of_visit"   "category"       
 [9] "score"           "visit_date"

## 6. Convert height units (mm → cm)

Right now height is stored as:

- `height_raw` – numeric value
- `height_unit_raw` – unit (`"cm"` or `"mm"`)

Having different units in the same column is dangerous because it can silently break analyses.

We'll create a clean `height_cm` variable where:

- If `height_unit_raw == "cm"`, we keep the value as is
- If `height_unit_raw == "mm"`, we divide by 10 to convert to centimeters
- For any unexpected unit, we set `height_cm` to `NA`


In [7]:
# 6. Unit conversion: height to centimeters ------------------------------

patients_step6 <- patients_step5 %>%
  mutate(
    height_cm = case_when(
      height_unit_raw == "cm" ~ height_raw,
      height_unit_raw == "mm" ~ height_raw / 10,
      TRUE                    ~ NA_real_
    )
  )

patients_step6 %>%
  select(id, height_raw, height_unit_raw, height_cm) %>%
  head(10)


id,height_raw,height_unit_raw,height_cm
<int>,<dbl>,<chr>,<dbl>
2,167,cm,167.0
4,1705,mm,170.5
7,164,cm,164.0
9,1740,mm,174.0
10,1506,mm,150.6
11,158,cm,158.0
12,173,cm,173.0
14,157,cm,157.0
18,157,cm,157.0


## 7. Final cleaned dataset

We now have a cleaned version of the dataset that includes:

- Missing values handled with simple imputation
- Datatypes fixed (e.g. `id` integer, `category` factor, `age` numeric)
- Dates parsed into a usable `visit_date` column
- Renamed height columns (`height_raw`, `height_unit_raw`)
- Height consistently expressed in centimeters (`height_cm`)

We'll remove redundant rows and store this in `patients_clean` and optionally write it out to a new CSV file.


In [8]:
# 7. Final cleaned data --------------------------------------------------

patients_clean <- patients_step6 %>%
    select(- height_raw, -height_unit_raw, -date_of_visit)
    

glimpse(patients_clean) 


# Optional: save cleaned data
# write_csv(patients_clean, "simple_dummy_data_clean.csv")


Rows: 15
Columns: 8
$ id         <int> 2, 4, 7, 9, 10, 11, 12, 14, 18, 19, 20, 24, 26, 27, 30
$ name       <chr> "Bob", "David", "Grace", "Ian", "Jane", "Karl", "Liam", "No…
$ age        <dbl> 28, 28, 29, 29, 29, 44, 45, 36, 31, 26, 37, 19, 35, 27, 42
$ weight_kg  <dbl> 81, 86, 56, 55, 46, 60, 77, 76, 86, 66, 81, 66, 72, 82, 81
$ category   <fct> B, B, A, B, C, A, B, B, A, C, B, B, A, A, B
$ score      <dbl> 42, 41, 48, 200, 42, 52, 46, 47, 58, 56, 45, 46, 54, 47, 56
$ visit_date <date> 2025-01-17, 2025-01-16, 2025-01-19, 2025-01-09, 2025-01-18,…
$ height_cm  <dbl> 167.0, 170.5, 164.0, 174.0, 150.6, 158.0, 173.0, 157.0, 157…


## YAYYYYY!!!!
### you made it to the end of this workshop! good luck with your project <3